 
 
     3Homework solution:    ML02_EDA
                   By:    K Dehnad

In [13]:
# pip install ydata-profiling --user

In [14]:
# !pip install numpy==2.0.2

In [15]:
!python --version


Python 3.12.6


In [16]:
!pip list


Package            Version
------------------ -----------
annotated-types    0.7.0
asttokens          3.0.0
attrs              25.1.0
certifi            2025.1.31
charset-normalizer 3.4.1
colorama           0.4.6
comm               0.2.2
contourpy          1.3.1
cycler             0.12.1
dacite             1.9.2
debugpy            1.8.12
decorator          5.2.1
executing          2.2.0
fonttools          4.56.0
htmlmin            0.1.12
idna               3.10
ImageHash          4.3.1
ipykernel          6.29.5
ipython            8.32.0
jedi               0.19.2
Jinja2             3.1.5
joblib             1.4.2
jupyter_client     8.6.3
jupyter_core       5.7.2
kiwisolver         1.4.8
MarkupSafe         3.0.2
matplotlib         3.10.0
matplotlib-inline  0.1.7
multimethod        1.12
nest-asyncio       1.6.0
networkx           3.4.2
numpy              2.0.2
packaging          24.2
pandas             2.2.3
parso              0.8.4
patsy              1.0.1
phik               0.12.4
pillow

In [17]:
# pip install pydantic-settings

In [18]:
# pip install ydata-profiling

In [ ]:
# from ydata_profiling import ProfileReport

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport


Iris_modified data dictionary 
SepalLength     Int64
SepalWidth      Int64
PetalLength     Int64
PetalWidth      Int64
Species         str

In [28]:
# Load the data into a pandas dataframe

coltype={'SepalLength':'Int64' , 'SepalWidth':'Int64','PetalLength':'Int64', 'PetalWidth':'Int64', 'Species':'str' }
folder='C:/Users/Asus/OneDrive/Stevens/Sem-2/KDDM/HW2/'

df = pd.read_csv(folder+'breast-cancer-wisconsin.data.csv',na_values='?',dtype=coltype)


df.head()
# df.tail(10)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Asus/OneDrive/Stevens/Sem-2/KDDM/HW2/breast-cancer-wisconsin.data.csv'

In [ ]:
print(df.dtypes)

In [ ]:


# Compute the minimum, maximum, and mean of each numeric column

# Summarize each column
summary = df.describe()

# Print the summary
print(summary)

print('\n min, max, mean only')
# Min_Max_Mean=df.describe(include=[float, int]).loc[["min", "max", "mean"]]
Min_Max_Mean=df.describe().loc[["min", "max", "mean"]]
print(Min_Max_Mean)



In [ ]:

profile = ProfileReport(df, title="Profiling data frame")
profile

In [ ]:
print(df[df.isna().any(axis=1)]) # show the na in the data frame
# print(df.isnull())
missing_values =df.isnull().sum()
print(missing_values)
 

In [ ]:


# # # Replace missing values with the mean of the column
# df['PetalLength'].fillna(df['PetalLength'].mean().round(),inplace=True)
df['PetalLength']=df['PetalLength'].fillna(df['PetalLength'].mean().round())

# df.fillna(df.mean(), inplace=True)
print("\n check for missing value again")
print(df.isnull().sum())
print(df[df.isna().any(axis=1)])


In [ ]:
#IV.	Displaying the frequency table of “PetalLength” vs. Species

freq_table = pd.crosstab(df['PetalLength'], df['Species'])
print(freq_table)

In [1]:
%matplotlib inline

In [ ]:
# # Displaying the scatter plot of  	SepalLength	SepalWidth	PetalLength	PetalWidth	Species, one pair at a time
# # Create scatter plot matrix

pd.plotting.scatter_matrix(df[['SepalLength','SepalWidth',  'PetalLength', 'PetalWidth']], figsize=(12,12), diagonal='hist')
# pd.plotting.scatter_matrix(df.loc[:, 'SepalLength':'PetalWidth'], figsize=(12,12), diagonal='hist')

# Show plot
plt.show()

In [ ]:
# Create a box plot
df[['SepalLength','SepalWidth',  'PetalLength', 'PetalWidth']].plot(kind="box", vert=True, figsize=(10, 6))
# subset=df.loc[:, 'SepalLength':'SepalWidth']
# subset.plot(kind="box", vert=True, figsize=(10, 6))
plt.title("Box plot", fontsize=12)
plt.xlabel("Value")
plt.show()

In [ ]:
df2=df.drop('Species',axis=1)
df2.head()
Q1 = df2.quantile(0.25)
Q3 = df2.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = (df2 < lower_bound) | (df2 > upper_bound)
df2['Outlier']=outliers.any(axis=1)
df2.head()
filtered_df = df2[df2['Outlier'] == True]  
print(filtered_df)

In [ ]:
# develop your regresion model
from statsmodels.formula.api import ols
res = ols(
    ' PetalWidth ~ SepalLength',
    data=df,
).fit()

In [ ]:
 # develop prediction for the original predictors
predictions = res.get_prediction(df['SepalLength'])
predict_df = predictions.summary_frame(alpha=.05)
predict_df.head()

In [ ]:
outliers = (df['PetalWidth'] < predict_df['obs_ci_lower']) | (df['PetalWidth'] > predict_df['obs_ci_upper'])
df['outlier'] = outliers
df.tail(20)

In [ ]:



# # Plot results
plt.figure(figsize=(8, 6))
plt.scatter(df['SepalLength'], df['PetalWidth'], label='df')
plt.plot(df['SepalLength'], predict_df['mean'] , color='black', label='Regression Line')
plt.plot(df['SepalLength'], predict_df['obs_ci_lower'] , color='green', label='Regression Line')
plt.plot(df['SepalLength'], predict_df['obs_ci_upper'] , color='green', label='Regression Line')
plt.scatter(df.loc[outliers, 'SepalLength' ], df.loc[outliers,'PetalWidth'  ], color='red', label='Outliers', edgecolors='k', s=50)
plt.xlabel('SepalLength')
plt.ylabel('PetalWidth')
# plt.legend()
# plt.title('Regression with 95% Confidence Interval and Outliers')
plt.show()

# Print identified outliers
print("Outliers:")
print(df.loc[outliers])